<a href="https://colab.research.google.com/github/adityavinodh/masters-projects/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8" # this is needed to get rid of weird colab locale error
# if you are still running into issues, please restart the runtime to initialize a new environment

!wget https://zenodo.org/records/7908468/files/python.zip
!unzip python.zip
!gzip -d python/final/jsonl/train/python_train_0.jsonl.gz

--2024-10-17 16:26:49--  https://zenodo.org/records/7908468/files/python.zip
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.98.238, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940909997 (897M) [application/octet-stream]
Saving to: ‘python.zip’

python.zip           16%[==>                 ] 148.31M  20.5MB/s    eta 45s    ^C
Archive:  python.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of python.zip or
        python.zip.zip, and cannot find python.zip.ZIP, period.
gzip: python/final/jsonl/train/python_train_0.jsonl.gz: No such file or directory


In [2]:
import numpy as np
import json

from google.colab import drive
drive.mount('/content/drive')
# Assuming your file is in your current working directory, adjust as needed
train_file = '/content/drive/MyDrive/Colab Notebooks/python/final/jsonl/train/python_train_0.jsonl'


def grab_raw_dataset():
    raw_dataset = []
    file = train_file
    with open(file, "r") as f:
        raw_dataset.extend([json.loads(x) for x in f.readlines()])
    print("grabbed data from file {}".format(file))
    print("Number of raw functions: {}".format(len(raw_dataset)))
    return raw_dataset

raw_dataset = grab_raw_dataset()

Mounted at /content/drive
grabbed data from file /content/drive/MyDrive/Colab Notebooks/python/final/jsonl/train/python_train_0.jsonl
Number of raw functions: 30000


In [3]:
import torch

from torch.utils.data import DataLoader
from transformers import AdamW, AutoTokenizer, T5ForConditionalGeneration

def load_base_model():
    tokenizer = AutoTokenizer.from_pretrained("t5-small")
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    return model, tokenizer

model, tokenizer = load_base_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def prepare_dataset(raw_dataset, tokenizer, mask_rate=0.15):
    input_ids_list = []
    labels_list = []

    for sample in raw_dataset:
        # Ensure each sample is a string
        if not isinstance(sample, str):
            sample = str(sample)

        # Tokenize with dynamic padding and truncation
        tokenized_input = tokenizer(sample, return_tensors="pt", padding="longest", truncation=True)

        input_ids = tokenized_input['input_ids']

        # Apply token masking
        masked_input_ids, labels = mask_tokens(input_ids, tokenizer, mask_rate)

        # Append the processed input_ids and labels
        input_ids_list.append(masked_input_ids)
        labels_list.append(labels)

        # Optionally: save intermediate results to disk if working with large datasets
        if len(input_ids_list) % 1000 == 0:
            # Save checkpoint to disk to free up memory
            save_intermediate_data(input_ids_list, labels_list)

    # Concatenate the tensors once after processing
    input_ids_tensor = torch.cat(input_ids_list, dim=0)
    labels_tensor = torch.cat(labels_list, dim=0)

    # Package input_ids and labels into encodings
    encodings = {"input_ids": input_ids_tensor, "labels": labels_tensor}

    # Return the dataset
    dataset = Dataset(encodings)
    return dataset


In [ ]:
# obtain the training dataset
training_dataset = prepare_dataset(raw_dataset=raw_dataset, tokenizer=tokenizer)

NameError: name 'mask_tokens' is not defined

In [ ]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

# Assuming `trained_dataset` is the dataset you want to save
save_path = '/content/drive/MyDrive/Colab Notebooks/python/final/jsonl/train/trained_dataset.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(training_dataset, f)

print(f"Dataset saved to {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: name 'training_dataset' is not defined

In [17]:
import pickle
import torch
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import AdamW

from google.colab import drive
drive.mount('/content/drive')
# Load the saved dataset
dataset_path = '/content/drive/MyDrive/Colab Notebooks/python/final/jsonl/train/trained_dataset.pkl'

with open(dataset_path, 'rb') as f:
    training_dataset = pickle.load(f)

print(f"Dataset loaded from {dataset_path}")

# Define the train function
def train(model, train_dataset, lr, batch_size, num_epochs):

    optimizer = AdamW(model.parameters(), lr=lr)

    if torch.cuda.is_available():
        print("cuda is available")
        model.cuda()

    loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    total_losses = []
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for input_ids, labels in tqdm(loader):
            input_ids = input_ids.cuda()
            labels = labels.cuda()

            outputs = model(input_ids=input_ids, labels=labels)

            loss = outputs.loss
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

    model.eval()
    return model


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


AttributeError: Can't get attribute 'Dataset' on <module '__main__'>

In [ ]:
batch_size = 16
learning_rate = 1e-4
num_epochs = 1

trained_model = train(model, training_dataset, lr=learning_rate, batch_size=batch_size, num_epochs=num_epochs, )

NameError: name 'train' is not defined

# Inference

In [8]:
!wget https://raw.githubusercontent.com/uiuc-cs598lmz-s24/hw1/main/hw_1_inference_dataset.jsonl

--2024-10-17 17:47:08--  https://raw.githubusercontent.com/uiuc-cs598lmz-s24/hw1/main/hw_1_inference_dataset.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1263405 (1.2M) [text/plain]
Saving to: ‘hw_1_inference_dataset.jsonl’

hw_1_inference_data 100%[===================>]   1.20M  --.-KB/s    in 0.03s   

2024-10-17 17:47:08 (34.5 MB/s) - ‘hw_1_inference_dataset.jsonl’ saved [1263405/1263405]



In [4]:
# reload the base model again
base_model, _ = load_base_model()

In [5]:
from google.colab import drive
drive.mount('/content/drive')
trained_model = '/content/drive/MyDrive/Colab Notebooks/python/final/jsonl/train/trained_model_full.pth'

# Define the model architecture (this must match your saved model)
model = T5ForConditionalGeneration.from_pretrained("t5-small")
trained_model = torch.load('/content/drive/MyDrive/Colab Notebooks/python/final/jsonl/train/trained_model_full.pth')


Mounted at /content/drive


<ipython-input-5-8648094c4d9e>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained_model = torch.load('/content/drive/MyDrive/Colab Notebooks/python/final/jsonl/train/t

In [11]:
import json

def grab_inference_dataset():
    inference_dataset = []
    drive.mount('/content/drive')
    # Load the saved dataset
    file = '/content/drive/MyDrive/Colab Notebooks/python/final/jsonl/hw_3_inference_dataset.jsonl'
    with open(file, "r") as f:
        inference_dataset.extend([json.loads(x) for x in f.readlines()])
    print("grabbed data from file {}".format(file))
    print("Number of tasks: {}".format(len(inference_dataset)))
    return inference_dataset

inference_dataset = grab_inference_dataset()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
grabbed data from file /content/drive/MyDrive/Colab Notebooks/python/final/jsonl/hw_3_inference_dataset.jsonl
Number of tasks: 1000


In [2]:
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.8 MB/s eta 0:00:00


In [6]:
def code_infill(code_model, tokenizer, prefix, suffix):
    # Define the placeholder token used in prepare_dataset
    mask_token = tokenizer.mask_token

    # Combine prefix and suffix with the mask token
    input_text = f"{prefix} {mask_token} {suffix}"

    # Tokenize the input
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)

    # Move inputs to the appropriate device
    inputs = {k: v.to(code_model.device) for k, v in inputs.items()}

    # Generate output using greedy decoding
    with torch.no_grad():
        outputs = code_model.generate(inputs['input_ids'], max_length=512, temperature=0.0)

    # Decode the output IDs to get the generated text
    model_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return model_output  # Return the raw model output instead of filled_code


In [12]:
from Levenshtein import distance as levenshtein_distance

def evaluate(code_model, tokenizer, inference_dataset):
    def edit_distance(model_output, gt):
        # Calculate the Levenshtein distance between model output and ground truth
        return levenshtein_distance(model_output, gt)

    ed_scores = []  # List to store edit distance scores
    for data in inference_dataset:
        prefix = data['prefix']
        suffix = data['suffix']
        gt = data['gt']

        # Use the code_infill function to generate the model's output
        model_output = code_infill(code_model, tokenizer, prefix, suffix)

        # Compute the edit distance and append to the scores list
        ed_scores.append(edit_distance(model_output, gt))

    # Calculate and return the average edit distance score
    import statistics as st
    return st.mean(ed_scores)

# Report the results
trained_model_ed = evaluate(trained_model, tokenizer, inference_dataset)

print(f"Trained Model Edit Distance: {trained_model_ed}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Trained Model Edit Distance: 227.059


In [14]:
import pickle

# Define the path where you want to save the variable (inside your Google Drive or Colab environment)
save_path = '/content/drive/MyDrive/Colab Notebooks/python/final/trained_model_ed.pkl'  # You can modify the path as needed

# Save the variable using pickle
with open(save_path, 'wb') as f:
    pickle.dump(trained_model_ed, f)

print(f"trained_model_ed saved at {save_path}")


trained_model_ed saved at /content/drive/MyDrive/Colab Notebooks/python/final/trained_model_ed.pkl


In [15]:
base_model_ed = evaluate(base_model, tokenizer, inference_dataset)

print(f"Base Model Edit Distance: {base_model_ed}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Base Model Edit Distance: 239.722


In [16]:
import pickle

# Define the path where you want to save the variable (inside your Google Drive or Colab environment)
save_path = '/content/drive/MyDrive/Colab Notebooks/python/final/base_model_ed.pkl'  # You can modify the path as needed

# Save the variable using pickle
with open(save_path, 'wb') as f:
    pickle.dump(base_model_ed, f)

print(f"base_model_ed saved at {save_path}")

base_model_ed saved at /content/drive/MyDrive/Colab Notebooks/python/final/base_model_ed.pkl
